# Test Metrics

## Import and Configuration

In [1]:
import psnr, rmse, vif, fsim, ssim, msssim
import numpy as np
import skimage.io as io
from skimage.color import rgb2gray
import torch

In [2]:
file_path_img_r = '../../TID2013/reference_images/I01.bmp'
file_path_img_m = '../../TID2013/distorted_images/I01_01_1.bmp'

img_r = io.imread(file_path_img_r)
img_m = io.imread(file_path_img_m)

img_r_gray = rgb2gray(img_r)
img_m_gray = rgb2gray(img_m)

img_r_tensor = torch.tensor(img_r).permute(2, 0, 1).unsqueeze(0)
img_m_tensor = torch.tensor(img_m).permute(2, 0, 1).unsqueeze(0)

In [3]:
img_r.shape

(384, 512, 3)

In [4]:
img_r_tensor.shape

torch.Size([1, 3, 384, 512])

## Calculations

### PSNR

In [7]:
metric = psnr.PSNR(data_range=255, normalize=False, batch=False)
metric.score(img_r, img_m)
metric.score_val

33.07421268229845

--> code OK, value OK

### RMSE

In [8]:
metric = rmse.RMSE(data_range=255, normalize=False)
metric.score(img_r, img_m)
metric.score_val

5.660170927248818

--> code OK, value probably OK

### VIFp

In [18]:
metric = vif.VIFp(data_range=255, normalize=False, chromatic=True)
metric.score(img_r, img_m)
metric.score_val

tensor(0.6713)

--> code OK, value **NOT** OK

In [20]:
# sewar VIFp
from sewar import vifp
print(vifp(img_r, img_m, sigma_nsq=3.2))
# piq VIFp
from piq import vif_p as piq_vif_p
print(piq_vif_p(img_r_tensor, img_m_tensor, data_range=255, sigma_n_sq=3.2))

0.6092888086704665
tensor(0.7071)


### FSIM

In [23]:
metric_fsim_m = fsim.FSIM(data_range=255, normalize=False, chromatic=True)
metric_fsim_m.score(img_r, img_m)
metric_fsim_m.score_val

tensor(0.9938)

--> code OK, value OK

### SSIM

In [30]:
metric_ssim_m = ssim.SSIM(data_range=255, normalize=False)
metric_ssim_m.score(img_r, img_m, gaussian_weights=True, use_sample_covariance=False, sigma=1.5, channel_axis=2)
metric_ssim_m.score_val

0.9167259591896975

--> code OK, value **NOT** OK

In [31]:
# sewar ssim
from sewar import ssim
print(ssim(img_r, img_m))

# piq ssim
from piq import ssim as piq_ssim
print(piq_ssim(img_r_tensor, img_m_tensor, data_range=255))

# skimage ssim
from skimage.metrics import structural_similarity as skimage_ssim
print(skimage_ssim(img_r, img_m, data_range=255, gaussian_weights=True, use_sample_covariance=False, sigma=1.5, channel_axis=2))

# pytorch-msssim ssim
from pytorch_msssim import ssim as pytorch_msssim_ssim
print(pytorch_msssim_ssim(img_r_tensor, img_m_tensor, data_range=255))

# pytorch-ignite ssim
from ignite.metrics import SSIM
metric = SSIM(data_range=255)
metric.update((img_r_tensor, img_m_tensor))
print(metric.compute())

(0.9369159676757867, 0.9369360732793357)
tensor(0.9784)
0.9167259591896975
tensor(1.)
0.9139210249821998


### MSSSIM

In [36]:
metric_msssim_m = msssim.MSSSIM(data_range=255, normalize=False)
metric_msssim_m.score(img_r, img_m)
metric_msssim_m.score_val

(0.9903786114953346+0j)

--> code **NOT** OK, value **NOT** OK

In [33]:
# sewar msssim
from sewar import msssim
print(msssim(img_r, img_m))

# piq msssim
from piq import multi_scale_ssim as piq_msssim
print(piq_msssim(img_r_tensor, img_m_tensor, data_range=255))

# pytorch-msssim msssim
from pytorch_msssim import ms_ssim as pytorch_msssim_msssim
print(pytorch_msssim_msssim(img_r_tensor, img_m_tensor, data_range=255))

(0.9903786114953346+0j)
tensor(0.9880)


RuntimeError: "avg_pool2d" not implemented for 'Byte'

## Conclusion

| Metric | Code OK    | Value OK |
| --- |------------| --- |
| PSNR | OK         | OK |
| RMSE | OK         | OK |
| VIFp | OK         | **NOT** OK |
| FSIM | OK         | OK |
| SSIM | OK         | **NOT** OK |
| MSSSIM | **NOT** OK | **NOT** OK |